In [4]:
##%overwritefile
##%file:src/compile_with_groovyc.py
##%noruncode
    def compile_with_groovyc(self, source_filename, binary_filepath=None, cflags=None, ldflags=None,env=None,coptions=None,magics=None):
        # coptions = ['-b', '', '-cp', '', '-j'] + cflags
        outpath=os.path.dirname(source_filename)
        sf = os.path.basename(source_filename)
        binary_filename = sf.split(".")[0]
        index=-1
        if coptions==None:
            coptions=[]
        for s in coptions:
            index=index+1
            if s=='-d':
                outpath=coptions[index+1]
                if not outpath.startswith('-'):
                    #剔除 -d参数和值
                    outpath=coptions.pop(index+1)
                    coptions.pop(index)
            else:
                if binary_filepath!=None:
                    outpath=binary_filepath
        args = ['groovyc']+coptions+ ['-d', outpath]+[ source_filename]
        self.mymagics._logln(' '.join((' '+ str(s) for s in args)))
        binary_filename=os.path.join(outpath,binary_filename)
        return self.mymagics.create_jupyter_subprocess(args,env=env,magics=magics),binary_filename+".class",args


[MyPythonKernel124954] Info:src/compile_with_groovyc.py
[MyPythonKernel124954] Info:file /mnt/h/Jupyter/Myjupyter-kernel/kernel/Groovy/src/compile_with_groovyc.py created successfully


In [5]:
##%overwritefile
##%file:src/g_exec_groovyc_.py
##%noruncode
    def _exec_groovyc_(self,source_filename,magics):
        self.mymagics._write_to_stdout('Generating binary file\n')
        magics['status']='compiling'
        p,outfile,ccmd = self.compile_with_groovyc(
            source_filename, 
            None,
            self.mymagics.get_magicsSvalue(magics,'cflags'),
            self.mymagics.get_magicsSvalue(magics,'ldflags'),
            self.mymagics.get_magicsbykey(magics,'env'),
            self.mymagics.get_magicsSvalue(magics,'coptions'),
            magics
            )
        returncode=p.wait_end(magics)
        p.write_contents()
        magics['status']=''
        if returncode != 0:  # Compilation failed
            self.mymagics._logln(''.join((str(s) for s in ccmd)),3)
            self.mymagics._logln("Groovyc exited with code {}, the executable will not be executed".format(returncode),3)

            # delete source files before exit
            os.remove(source_filename)
            # os.remove(binary_file.name)
        return p.returncode,outfile


[MyPythonKernel124954] Info:src/g_exec_groovyc_.py
[MyPythonKernel124954] Info:file /mnt/h/Jupyter/Myjupyter-kernel/kernel/Groovy/src/g_exec_groovyc_.py created successfully
